In [ ]:
# default_exp type_emb
# all_data

In [ ]:
# hide
%load_ext autoreload
%autoreload 2

# Type embeddings

## Imports

In [ ]:
from socemb.read_data import *

In [ ]:
# export
from gensim.models import Word2Vec
import pandas as pd
import re
from gensim.models import KeyedVectors
from scipy.spatial.distance import cosine

## Variables

In [ ]:
VECS_DIR = 'data/vecs/'

In [ ]:
LIMIT = 100_000

In [ ]:
SUBREDDIT = 'askreddit'
T_1 = 2010
T_2 = 2020

In [ ]:
COMM_1 = 'askaconservative'
COMM_2 = 'politics'

In [ ]:
YEAR = 2020

## Train model

### Read comments

In [ ]:
f_paths = get_fpaths_yr(YEAR)

In [ ]:
comments = read_comm_csvs(f_paths)

In [ ]:
comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380964 entries, 0 to 380963
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   body         380964 non-null  string        
 1   created_utc  380964 non-null  datetime64[ns]
 2   id           380964 non-null  string        
 3   subreddit    380964 non-null  string        
dtypes: datetime64[ns](1), string(3)
memory usage: 11.6 MB


### Split data (inactive)

### Split in temporal bins

In [ ]:
comments = comments.assign(date = pd.to_datetime(
    comments['created_utc'],
    errors='coerce'
))

In [ ]:
comments['date'].dt.year.value_counts()

In [ ]:
comments = comments[comments.date.dt.year == TIME]

### Split by communities

In [ ]:
comments\
    .value_counts('subreddit')\
    .head(20)

In [ ]:
comments = comments.query('subreddit == "politics"')

In [ ]:
comments = comments.query('subreddit == "AskHistorians"')

### Pre-processing

In [ ]:
# export
def conv_to_lowerc(doc):
    return doc.lower()

In [ ]:
assert conv_to_lowerc('Test') == 'test'

In [ ]:
# export
def rm_punct(doc):
    return re.sub(r'[^\w\s]+', ' ', doc)

In [ ]:
assert rm_punct('No-punctuation!') == 'No punctuation '

In [ ]:
# export
def tokenize(doc):
    return doc.split()

In [ ]:
assert len(tokenize('These are three-tokens')) == 3

In [ ]:
# export
def detect_short_doc(doc, limit=10):
    if len(doc) >= limit:
        return False
    else:
        return True

In [ ]:
assert detect_short_doc(['oans', 'zwoa', 'drei'], 10) == True

In [ ]:
assert detect_short_doc(['oans', 'zwoa', 'drei', 'viere', 'fuenfe', 'sechse', 'simme', 'achte', 'neine', 'zehne'], 10) == False

In [ ]:
# export
def clean_docs(docs):
    docs_clean = docs\
        .apply(conv_to_lowerc)\
        .apply(rm_punct)\
        .apply(tokenize)\
        .where(lambda x : x.apply(detect_short_doc) == False)\
        .dropna()    
    return docs_clean

In [ ]:
docs_clean = clean_docs(comments['body'])

In [ ]:
docs_clean

0         [you, re, asking, how, they, re, going, to, be...
1         [gt, i, don, t, think, there, are, any, varyin...
2         [its, split, on, copyright, it, leans, anti, n...
3         [that, would, be, up, to, the, land, owners, a...
4         [i, have, him, here, gun, to, his, head, round...
                                ...                        
380954    [the, nature, police, are, supposed, to, be, p...
380957    [i, got, reddit, last, year, and, only, starte...
380958    [i, m, just, curious, how, this, is, clear, to...
380962    [staying, at, said, job, while, being, underpa...
380963    [i, m, not, sure, that, this, has, ever, been,...
Name: body, Length: 264796, dtype: object

### Lexeme-based

In [ ]:
docs_clean = docs_clean\
    .str.replace('anglo-saxon', 'anglosaxon')\
    .str.replace('anglo saxon', 'anglosaxon')

In [ ]:
docs_clean = docs_clean.str.replace('anglosaxon', 'Anglo-Saxon')

### Create corpus

In [ ]:
# export
class Corpus:
    """An iterator that yields sentences (lists of str)."""
    def __init__(self, docs_clean):
        self.docs_clean = docs_clean

    def __iter__(self):
        for doc in self.docs_clean:
            yield doc

In [ ]:
corpus = Corpus(docs_clean)

### Train embeddings

In [ ]:
# export
def train_emb(corpus, MIN_COUNT=5, SIZE=300, WORKERS=8, WINDOW=3):
    model = Word2Vec(
        corpus, 
        min_count=MIN_COUNT,
        size=SIZE,
        workers=WORKERS, 
        window=WINDOW
    )
    return model

In [ ]:
%%time
model = train_emb(corpus)

CPU times: user 2min 35s, sys: 11.1 s, total: 2min 46s
Wall time: 1min 9s


In [ ]:
wv = model.wv

In [ ]:
wv['you'].shape

(300,)

### Evaluate embeddings

In [ ]:
for index, word in enumerate(wv.index2word):
    if index == 10:
        break
    print(f"{index}/{len(wv.index2word)}: {word}")

0/4732: the
1/4732: to
2/4732: a
3/4732: and
4/4732: i
5/4732: of
6/4732: that
7/4732: it
8/4732: is
9/4732: you


In [ ]:
wv.most_similar('person', topn=20)

[('guy', 0.6144881248474121),
 ('woman', 0.6049772500991821),
 ('man', 0.5703142881393433),
 ('politician', 0.5600118637084961),
 ('girl', 0.5361282229423523),
 ('candidate', 0.5117915868759155),
 ('cop', 0.5107803344726562),
 ('someone', 0.4916432201862335),
 ('senator', 0.4843234419822693),
 ('kid', 0.4767877161502838),
 ('persons', 0.46128833293914795),
 ('victim', 0.45970290899276733),
 ('somebody', 0.4551505148410797),
 ('lady', 0.4502111077308655),
 ('citizen', 0.44879597425460815),
 ('owner', 0.44654402136802673),
 ('mother', 0.44434842467308044),
 ('parent', 0.44392669200897217),
 ('fetus', 0.4342397451400757),
 ('entity', 0.43350768089294434)]

In [ ]:
lex_rel_1 = 'person'
lex_rel_2 = 'man'
lex_unrel = 'time'

In [ ]:
sim_rel = wv.similarity(lex_rel_1, lex_rel_2)
sim_rel

0.57419455

In [ ]:
sim_unrel_1 = wv.similarity(lex_rel_1, lex_unrel)
sim_unrel_1

0.27401066

In [ ]:
sim_unrel_2 = wv.similarity(lex_rel_2, lex_unrel)
sim_unrel_2

0.14052157

In [ ]:
assert sim_rel > sim_unrel_1

In [ ]:
assert sim_rel > sim_unrel_2

## Save model

In [ ]:
wv.save(f'~/promo/socemb/data/vecs/year/{YEAR}.wv')

## Load models

In [ ]:
wv = KeyedVectors.load(f'{VECS_DIR}{SUBREDDIT}_{YEAR}_{LIMIT}.wv', mmap='r')

In the next cell, I need to to use `init_sims` to normalize the vectors. To do this, I need to save and load full models, not just the vector tables. Therefore, I need to train all models again.

In [ ]:
# export
def load_model(SUBREDDIT: str, YEAR: int, LIMIT: int = 100_000):
    """Load word2vec model from disk."""
    model = KeyedVectors.load(f'data/vecs/{SUBREDDIT}_{YEAR}_{LIMIT}.wv', mmap='r')
    return model

In [ ]:
YEARS = range(2013, 2021)

In [ ]:
YEAR = 2013

In [ ]:
LEX = 'trump'

In [ ]:
model_1 = load_model(COMM_1, YEAR)

In [ ]:
model_2 = load_model(COMM_2, YEAR)

## Extract vector

In [ ]:
# export
def get_vec_from_model(lex: str, model):
    return model[lex]

In [ ]:
get_vec_from_model(LEX, model_1)

array([ 1.00899458e-01, -1.52170643e-01, -9.39689055e-02, -9.99266282e-03,
        8.40886682e-02, -2.21601501e-02, -2.10636541e-01,  3.73399556e-02,
       -1.49149716e-01,  2.40157068e-01, -6.44295737e-02,  3.83358356e-03,
        1.28691554e-01, -3.57708037e-02,  5.21394191e-03, -1.38501897e-01,
       -5.26996851e-02,  1.61609109e-02, -8.78354255e-03,  7.47078285e-02,
        1.85247645e-01, -1.60785943e-01,  9.02396999e-03, -8.07802901e-02,
        1.38207108e-01,  7.45966509e-02, -8.67893994e-02, -3.15905828e-03,
       -9.56856683e-02,  1.56406348e-03, -1.31446823e-01, -1.16315879e-01,
       -1.51329443e-01,  3.41142602e-02, -2.11179614e-01,  1.42359763e-01,
        4.96119075e-02, -7.68923908e-02, -1.69393018e-01,  6.09558728e-03,
       -6.81157708e-02,  3.04796919e-02,  1.52811790e-02,  8.24113786e-02,
       -1.50099978e-01, -8.96327049e-02, -8.57340470e-02,  1.70979887e-01,
       -1.25112638e-01,  2.76306607e-02,  2.79491618e-02,  9.32807103e-02,
       -8.61876681e-02,  

In [ ]:
vec_1 = model_1['trump']

In [ ]:
vec_2 = model_2['trump']

Soc / 2020 / conservative vs. politics

In [ ]:
cosine(trump_cons_vec, trump_pol_vec)

0.6027644872665405

Soc / 2013 / conservative vs. politics

In [ ]:
cosine(vec_1, vec_2)

0.5683872997760773

## Measure distances

### For `1` pair of words

In [ ]:
vec_t1 = model_1['person']

In [ ]:
vec_t2 = model_2['person']

In [ ]:
assert vec_t1.shape[0] == 300

In [ ]:
assert vec_t2.shape[0] == 300

In [ ]:
cosine(vec_t1, vec_t2)

0.6892097592353821

### For full vocabulary

In [ ]:
def get_distances(model_1, model_2):
    dist_dict = {}
    # will need to replace `vecs_t1.vocab` with a general vocab
    for word in model_1.vocab:
        if word in model_2.vocab:
            dist_dict[word] = cosine(model_1[word], model_2[word])
        else:
            pass
    dist_dict_df = pd.DataFrame(
        data=dist_dict.items(), 
        columns=['word', 'dist']
    )\
        .sort_values('dist', ascending=False)
    return dist_dict_df

In [ ]:
dists = get_distances(model_1, model_2)

In [ ]:
dists.nlargest(20, 'dist')

,word,dist
415,2000,1.094130
8749,kentucky,1.061203
2872,mitch,1.040116
12751,irreparable,1.026567
12012,2k,1.025281
5045,r,1.010832
706,600,1.001934
484,georgia,0.998421
11903,moscow,0.993207
4954,mcconnell,0.990447


### Inspect nearest neighbours

In [ ]:
lex = 'moscow'

In [ ]:
for model in [model_1, model_2]:
    for nb, dist in model.most_similar(lex, topn=10):
        print(f'{nb:<15}{model.vocab[nb].count:>6}')
    print('\n---\n')

tennessee          22
indiana            30
boston             40
massachusetts      41
hawaii             33
philly             30
hogan              21
mississippi        26
1989                8
nv                 13

---

mcconnell        3447
mcconnel          171
mcconnells         91
bluffing            6
mconnell           11
mcconell           18
mcturtle           35
blocks            101
mitch            3221
turtle            249

---



In [ ]:
for model in [model_1, model_2]:
    i = 0
    while i < 10:
        for nb, dist in model.most_similar(lex):
            if model.vocab[nb].count > 300:
                print(f'{nb:<15}{model.vocab[nb].count:>6}')
                i += 1
    print('\n---\n')

KeyboardInterrupt: 

### Temporal variation

In [ ]:
lex = 'trump'

In [ ]:
for model in [model_1, model_2]:
    for nb, dist in model.most_similar(lex, topn=10):
        print(nb)
    print('\n---\n')

bankrupt
absorb
distributed
detect
essentials
composite
bypass
ideally
glide
prop

---

biden
bernie
obama
hillary
sanders
he
himself
nixon
putin
pelosi

---



### Social variation

In [ ]:
lex = 'quarantine'

In [ ]:
print(f'{COMM_1}\n---')
for nb, dist in model_1.most_similar(lex, topn=10):
    print(nb)

askaconservative
---
ppe
rain
bombings
port
aerosols
bullets
communications
missiles
weekly
2006


In [ ]:
print(f'{COMM_2}\n---')
for nb, dist in model_2.most_similar(lex, topn=10):
    print(nb)

asklibertarians
---
defense
defence
righteous
sufficiency
determination
isolate
legally
agency
permitted
explanatory


## Align models

In [ ]:
import numpy as np
import gensim

In [ ]:
smart_procrustes_align_gensim(model_1, model_2)

<ipython-input-226-3929bd79b83f>:33: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.vectors_norm instead).
  old_arr = m.syn0norm


IndexError: index 18164 is out of bounds for axis 0 with size 17915

In [ ]:
def smart_procrustes_align_gensim(base_embed, other_embed, words=None):
	"""Procrustes align two gensim word2vec models (to allow for comparison between same word across models).
	Code ported from HistWords <https://github.com/williamleif/histwords> by William Hamilton <wleif@stanford.edu>.
		(With help from William. Thank you!)
	First, intersect the vocabularies (see `intersection_align_gensim` documentation).
	Then do the alignment on the other_embed model.
	Replace the other_embed model's syn0 and syn0norm numpy matrices with the aligned version.
	Return other_embed.
	If `words` is set, intersect the two models' vocabulary with the vocabulary in words (see `intersection_align_gensim` documentation).
	"""
	
	# patch by Richard So [https://twitter.com/richardjeanso) (thanks!) to update this code for new version of gensim
	base_embed.init_sims()
	other_embed.init_sims()

	# make sure vocabulary and indices are aligned
	in_base_embed, in_other_embed = intersection_align_gensim(base_embed, other_embed, words=words)

	# get the embedding matrices
	base_vecs = in_base_embed.syn0norm
	other_vecs = in_other_embed.syn0norm

	# just a matrix dot product with numpy
	m = other_vecs.T.dot(base_vecs) 
	# SVD method from numpy
	u, _, v = np.linalg.svd(m)
	# another matrix operation
	ortho = u.dot(v) 
	# Replace original array with modified one
	# i.e. multiplying the embedding matrix (syn0norm)by "ortho"
	other_embed.syn0norm = other_embed.syn0 = (other_embed.syn0norm).dot(ortho)
	return other_embed
	

In [ ]:
def intersection_align_gensim(m1,m2, words=None):
	"""
	Intersect two gensim word2vec models, m1 and m2.
	Only the shared vocabulary between them is kept.
	If 'words' is set (as list or set), then the vocabulary is intersected with this list as well.
	Indices are re-organized from 0..N in order of descending frequency (=sum of counts from both m1 and m2).
	These indices correspond to the new syn0 and syn0norm objects in both gensim models:
		-- so that Row 0 of m1.syn0 will be for the same word as Row 0 of m2.syn0
		-- you can find the index of any word on the .index2word list: model.index2word.index(word) => 2
	The .vocab dictionary is also updated for each model, preserving the count but updating the index.
	"""

	# Get the vocab for each model
	vocab_m1 = set(m1.vocab.keys())
	vocab_m2 = set(m2.vocab.keys())

	# Find the common vocabulary
	common_vocab = vocab_m1&vocab_m2
	if words: common_vocab&=set(words)

	# If no alignment necessary because vocab is identical...
	if not vocab_m1-common_vocab and not vocab_m2-common_vocab:
		return (m1,m2)

	# Otherwise sort by frequency (summed for both)
	common_vocab = list(common_vocab)
	common_vocab.sort(key=lambda w: m1.vocab[w].count + m2.vocab[w].count,reverse=True)

	# Then for each model...
	for m in [m1,m2]:
		# Replace old syn0norm array with new one (with common vocab)
		indices = [m.vocab[w].index for w in common_vocab]
		old_arr = m.syn0norm
		new_arr = np.array([old_arr[index] for index in indices])
		m.syn0norm = m.syn0 = new_arr

		# Replace old vocab dictionary with new one (with common vocab)
		# and old index2word with new one
		m.index2word = common_vocab
		old_vocab = m.vocab
		new_vocab = {}
		for new_index,word in enumerate(common_vocab):
			old_vocab_obj=old_vocab[word]
			new_vocab[word] = gensim.models.word2vec.Vocab(index=new_index, count=old_vocab_obj.count)
		m.vocab = new_vocab

	return (m1,m2)

In [ ]:
m1 = model_1
m2 = model_2

In [ ]:
vocab_m1 = set(m1.vocab.keys())
vocab_m2 = set(m2.vocab.keys())

In [ ]:
common_vocab = vocab_m1 & vocab_m2

In [ ]:
common_vocab = list(common_vocab)
common_vocab.sort(key=lambda w: m1.vocab[w].count + m2.vocab[w].count,reverse=True)

Note that for the new Gensim versions, calls for .index2word, .vocab, .syn0 and .syn0norm should be replaced with .wv.index2word, .wv.vocab, .wv.syn0 and .wv.syn0norm respectively.

In [ ]:
for m in [m1,m2]:
    # Replace old syn0norm array with new one (with common vocab)
    indices = [m.vocab[w].index for w in common_vocab]
    old_arr = m.syn0norm
    new_arr = np.array([old_arr[index] for index in indices])
    m.syn0norm = m.syn0 = new_arr

<ipython-input-255-02a3945973d7>:4: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.vectors_norm instead).
  old_arr = m.syn0norm


IndexError: index 18164 is out of bounds for axis 0 with size 17915

<ipython-input-254-e3f9c0e261ee>:2: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.vectors_norm instead).
  m.syn0norm = m.syn0 = new_arr
<ipython-input-254-e3f9c0e261ee>:2: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  m.syn0norm = m.syn0 = new_arr


In [ ]:
    # Replace old vocab dictionary with new one (with common vocab)
    # and old index2word with new one
    m.index2word = common_vocab
    old_vocab = m.vocab
    new_vocab = {}
    for new_index,word in enumerate(common_vocab):
        old_vocab_obj=old_vocab[word]
        new_vocab[word] = gensim.models.word2vec.Vocab(index=new_index, count=old_vocab_obj.count)
    m.vocab = new_vocab

In [ ]:
def align_gensim_models(models, words=None):
    """
    Returns the aligned/intersected models from a list of gensim word2vec models.
    Generalized from original two-way intersection as seen above.
    
    Also updated to work with the most recent version of gensim
    Requires reduce from functools
    
    In order to run this, make sure you run 'model.init_sims()' for each model before you input them for alignment.
    
    ##############################################
    ORIGINAL DESCRIPTION
    ##############################################
    
    Only the shared vocabulary between them is kept.
    If 'words' is set (as list or set), then the vocabulary is intersected with this list as well.
    Indices are re-organized from 0..N in order of descending frequency (=sum of counts from both m1 and m2).
    These indices correspond to the new syn0 and syn0norm objects in both gensim models:
        -- so that Row 0 of m1.syn0 will be for the same word as Row 0 of m2.syn0
        -- you can find the index of any word on the .index2word list: model.index2word.index(word) => 2
    The .vocab dictionary is also updated for each model, preserving the count but updating the index.
    """

    # Get the vocab for each model
    vocabs = [set(m.vocab.keys()) for m in models]

    # Find the common vocabulary
    common_vocab = reduce((lambda vocab1,vocab2: vocab1&vocab2), vocabs)
    if words: common_vocab&=set(words)

    # If no alignment necessary because vocab is identical...
    
    # This was generalized from:
    # if not vocab_m1-common_vocab and not vocab_m2-common_vocab and not vocab_m3-common_vocab:
    #   return (m1,m2,m3)
    if all(not vocab-common_vocab for vocab in vocabs):
        print("All identical!")
        return models
        
    # Otherwise sort by frequency (summed for both)
    common_vocab = list(common_vocab)
    common_vocab.sort(key=lambda w: sum([m.vocab[w].count for m in models]),reverse=True)
    
    # Then for each model...
    for m in models:
        
        # Replace old vectors_norm array with new one (with common vocab)
        indices = [m.vocab[w].index for w in common_vocab]
                
        old_arr = m.vectors_norm
                
        new_arr = np.array([old_arr[index] for index in indices])
        m.vectors_norm = m.syn0 = new_arr

        # Replace old vocab dictionary with new one (with common vocab)
        # and old index2word with new one
        m.index2word = common_vocab
        old_vocab = m.vocab
        new_vocab = {}
        for new_index,word in enumerate(common_vocab):
            old_vocab_obj=old_vocab[word]
            new_vocab[word] = gensim.models.word2vec.Vocab(index=new_index, count=old_vocab_obj.count)
        m.vocab = new_vocab

    return models

In [ ]:
from functools import reduce

In [ ]:
model_1.init_sims()
model_2.init_sims()

In [ ]:
align_gensim_models([model_1, model_2])

IndexError: index 18164 is out of bounds for axis 0 with size 17915

## Analysis

### Semantic change / neology

#### 2010 vs. 2020

In [ ]:
dists.nlargest(20, 'dist')

,word,dist
3368,theonion,1.073174
14398,trump,1.064995
24331,pronouns,1.063726
5941,quarantine,1.026087
20712,autonomy,1.000578
2834,chat,0.993065
19200,ccp,0.990134
3474,guardian,0.988905
23239,lockdown,0.987755
16618,230,0.987645


#### 'trump'

In [ ]:
for model in [model_1, model_2]:
    for nb, dist in model.most_similar(lex, topn=10):
        print(f'{nb:<15}{model.vocab[nb].count:>6}')
    print('\n---\n')

fatten              5
revert             19
seize              33
flare              14
tighten            20
spark              44
dissolve           31
freeze             64
stifle             32
clog                9

---

biden            8867
obama            3149
trumps            843
he              62307
putin             315
djt                70
hillary           896
bernie           2856
him             17557
jojo              110

---



## Obsolete stuff

### Gensim preprocessing